In [1]:
from __future__ import print_function
import os
from os import listdir
from os.path import join
import random
import logging
import time
import copy
from turtle import down
import gdown

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchinfo import summary
from tqdm.notebook import tqdm
from sklearn.metrics import roc_curve, auc, confusion_matrix

from models.cvt import CvT, EqCvT
from typing import *
from utils.util import (
    make_directories,
    seed_everything,
    get_device,
    init_logging_handler,
)
from utils.dataset import download_dataset, DeepLenseDataset, visualize_samples
from utils.train import train
from utils.inference import Inference
from argparse import ArgumentParser
from config.data_config import DATASET
from config.eqcvt_config import EQCVT_CONFIG
from utils.augmentation import get_transform_test, get_transform_train
from torch.utils.data import DataLoader
import timm
from torchvision import models

In [2]:
dataset_name = "Model_I"
dataset_dir = "data"
use_cuda = True

classes = DATASET[f"{dataset_name}"]["classes"]

train_config = EQCVT_CONFIG
network_type = train_config["network_type"]
network_config = train_config["network_config"]
image_size = train_config["image_size"]
optimizer_config = train_config["optimizer_config"]
lr_schedule_config = train_config["lr_schedule_config"]

make_directories([dataset_dir])

In [3]:
trainset = DeepLenseDataset(
    dataset_dir,
    "train",
    dataset_name,
    transform=get_transform_train(upsample_size=387, final_size=129, channels=3),
    download=True,
    channels=3
)  # get_transform_train()


Dataset already exists


In [4]:
testset = DeepLenseDataset(
    dataset_dir,
    "test",
    dataset_name,
    transform=get_transform_test(final_size=129, channels=3),
    download=True,
    channels=3
)  # transform_test

Dataset already exists


In [5]:
seed_everything(seed=42)
device = get_device(use_cuda=use_cuda, cuda_idx=0)

# logging
current_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
log_dir = "logger"
init_logging_handler(log_dir, current_time)

PATH = os.path.join(f"{log_dir}/checkpoint", f"{network_type}_{current_time}.pt")

Using device = cuda:0


In [6]:
train_loader = DataLoader(
    dataset=trainset, batch_size=train_config["batch_size"], shuffle=True
)
test_loader = DataLoader(
    dataset=testset, batch_size=train_config["batch_size"], shuffle=True
)

# visualize_samples(dataset=trainset, labels_map=classes)

In [7]:
num_classes = len(classes)  # number of classes to be classified
# image size (129x129)
print(num_classes)

3


In [8]:
print(f"Train Data: {len(trainset)}")
print(f"Val Data: {len(testset)}")

Train Data: 87525
Val Data: 15000


In [9]:
model = timm.create_model('resnet50d', pretrained=True)
summary(model)

INFO:timm.models.helpers:Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50d_ra2-464e36ba.pth)


Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       864
│    └─BatchNorm2d: 2-2                  64
│    └─ReLU: 2-3                         --
│    └─Conv2d: 2-4                       9,216
│    └─BatchNorm2d: 2-5                  64
│    └─ReLU: 2-6                         --
│    └─Conv2d: 2-7                       18,432
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-8                   --
│    │    └─Conv2d: 3-1                  4,096
│    │    └─BatchNorm2d: 3-2             128
│    │    └─ReLU: 3-3                    --
│    │    └─Conv2d: 3-4                  36,864
│    │    └─BatchNorm2d: 3-5             128
│    │    └─ReLU: 3-6                    --
│    │    └─Conv2d: 3-7                  16,384
│    

In [10]:
num_features = model.get_classifier().in_features
num_features

2048

In [11]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [12]:
model.fc = nn.Sequential(
    nn.BatchNorm1d(num_features),
    nn.Linear(in_features=num_features, out_features=512,bias=False),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Linear(in_features=512, out_features=num_classes,bias=False)
)

In [13]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
# model_ft = models.densenet121(pretrained=True)
set_parameter_requires_grad(model, True)
# num_ftrs = model.classifier.in_features
model.fc = nn.Linear(num_features, num_classes)
model.to(device)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [14]:
# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=optimizer_config["lr"], betas=optimizer_config["betas"], weight_decay=optimizer_config["weight_decay"])

# scheduler
step_lr = train_config["lr_schedule_config"]["step_lr"]
reduce_on_plateau = train_config["lr_schedule_config"]["reduce_on_plateau"]

scheduler_plateau = ReduceLROnPlateau(optimizer, 'min', factor=reduce_on_plateau["factor"], patience=reduce_on_plateau["patience"], threshold=reduce_on_plateau["threshold"], verbose=reduce_on_plateau["verbose"])
scheduler_step = StepLR(optimizer, step_size=step_lr["step_size"], gamma=step_lr["gamma"])


In [15]:
train(
    epochs = train_config["num_epochs"],
    model = model,
    device = device,
    train_loader = train_loader,
    valid_loader = test_loader, # change to val-loader
    criterion = criterion,
    optimizer = optimizer,
    use_lr_schedule = train_config["lr_schedule_config"]["use_lr_schedule"],
    scheduler_step = scheduler_step,
    path = PATH,
)

DEBUG:root:Epoch : 1 - LR 0.00010000 - loss : 1.0914 - val_loss : 1.0902 - val_acc: 0.3824 



FileNotFoundError: [Errno 2] No such file or directory: 'logger/checkpoint/eqcvt_2022-06-22-17-13-00.pt'